In [1]:
# Spark session and functions
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count

# Machine Learning imports
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName('CreditCardFraudDetection').getOrCreate()

24/11/05 18:13:55 WARN Utils: Your hostname, Nikhils-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.206 instead (on interface en0)
24/11/05 18:13:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/05 18:13:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Dataset
The dataset is available at [Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud).

In [3]:
df = spark.read.csv('dataset/creditcard.csv', header=True, inferSchema=True)

## Explaining the dataset
The dataset contains transactions made by credit cards in September 2013 by european cardholders. There are 28 features, all of them are numerical. The features V1, V2, ... V28 are the result of a PCA transformation. The only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.  
V1, V2, ... V28 are the principal components obtained with PCA. They have been anonymized to protect sensitive information.

In [4]:
df.printSchema()

root
 |-- Time: double (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable = true)
 |-- V23: double (nullable = true)
 |-- V24: double (nullable = true)
 |-- V25: double (nullable = true)
 |-- V26: double (nullable = true)
 |-- V27: double (nullable = true)
 |-- V28: double (nulla

# Preprocessing
## Checking for missing values
First, we check for missing values in the dataset. If there are any, we will clean them up.

In [5]:
# Checking for missing values
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

24/11/05 18:13:59 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|   0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+



In [6]:
original_size = df.count()
print(f'Original size: {original_size}')

# Clean up missing values
df_cleaned = df.na.drop()

cleaned_size = df_cleaned.count()
print(f'Cleaned size: {cleaned_size}')

Original size: 284807
Cleaned size: 284807


## Assembling the features
In PySpark MLlib, all input features should be combined into a single vector column. To do this, you use the `VectorAssembler`.  
This is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees.


In [8]:
# Select feature columns (all except 'Class', which is the label)
feature_columns = df_cleaned.columns[:-1]  # Assuming the last column is the label

# Assemble feature columns into a single vector column called "features"
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_transformed = assembler.transform(df_cleaned)

# Show the transformed data
df_transformed.select("features", "Class").show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## Scaling the features
Feature scaling is important because some machine learning algorithms are sensitive to the scale of input data. You can scale features using the `StandardScaler`.
StandardScaler transforms a dataset of Vector rows, normalizing each feature to have unit standard deviation and/or zero mean. It takes parameters:
- `inputCol`: input column name.
- `outputCol`: output column name.
- `withStd`: True by default. Scales the data to unit standard deviation.
- `withMean`: False by default. Centers the data with mean before scaling. It will build a dense output, so take care when applying to sparse input.

In [10]:
# Scale features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=False)
df_scaled = scaler.fit(df_transformed).transform(df_transformed)

# Summary of the scaling process
print(f"Mean: {scaler.explainParams()}")
# print(f"Standard Deviation: {scaler.std}")

# Show the scaled data
df_scaled.select("scaled_features", "Class").show(5)

Mean: inputCol: input column name. (current: features)
outputCol: output column name. (default: StandardScaler_a04f4c59bc97__output, current: scaled_features)
withMean: Center data with mean (default: False, current: False)
withStd: Scale to unit standard deviation (default: True, current: True)
+--------------------+-----+
|     scaled_features|Class|
+--------------------+-----+
|[0.0,-0.694241102...|    0|
|[0.0,0.6084952594...|    0|
|[2.10578867609768...|    0|
|[2.10578867609768...|    0|
|[4.21157735219537...|    0|
+--------------------+-----+
only showing top 5 rows



## Class distribution
We group the dataset by the 'Class' column to check the distribution of the classes.

In [9]:
# Checking for class distribution
df_cleaned.groupBy('Class').count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



## Handling Class imbalance
From the class distribution, we can see that the dataset is highly imbalanced. The number of non-fraudulent transactions is much higher than the number of fraudulent transactions. For our model to learn effectively, we need to balance the classes.  
This can be done in two ways:
1. **Undersampling**: In this method, we randomly sample the majority class to match the minority class. This can lead to loss of information.
2. **Oversampling**: In this method, we randomly duplicate the minority class to match the majority class. This can lead to overfitting.

With this dataset, we will use the undersampling method as the dataset is large enough to not lose much information.

In [12]:
# Count the number of fraudulent and non-fraudulent transactions
fraud_count = df_cleaned.filter(col('Class') == 1).count()
non_fraud_count = df_cleaned.filter(col('Class') == 0).count()
print(f"Fraudulent transactions: {fraud_count}\nNon-fraudulent transactions: {non_fraud_count} before balancing")

# Undersampling the non-fraudulent transactions
df_fraud = df_cleaned.filter(col('Class') == 1)
df_non_fraud = df_cleaned.filter(col('Class') == 0).sample(False, fraud_count/non_fraud_count)

# Combine the two classes
df_balanced = df_fraud.union(df_non_fraud)

# Count the number of fraudulent and non-fraudulent transactions after balancing
df_balanced.groupBy('Class').count().show()

Fraudulent transactions: 492
Non-fraudulent transactions: 284315 before balancing
+-----+-----+
|Class|count|
+-----+-----+
|    1|  492|
|    0|  450|
+-----+-----+



# Splitting the dataset
Now that we have completed the preprocessing steps, we can split the dataset into training and testing sets.